In [1]:
import shap
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
data=pd.read_csv("g1.csv")
data.head()
continuous_features = data.columns.tolist()
continuous_features

x = data.drop("gender", axis=1)


# Target (the 'gender' column)
y = data["gender"]



In [3]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=7)

target_feature = "gender"


train_data = X_train.copy()
test_data = X_test.copy()


train_data[target_feature] = y_train
test_data[target_feature] = y_test



feature_names = train_data.columns.tolist()
print("Feature names:", feature_names)



Feature names: ['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender']


In [4]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the Logistic Regression model
model = LogisticRegression(random_state=7)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix to visualize the performance
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9710

Classification Report:
              precision    recall  f1-score   support

      Female       0.98      0.97      0.97       497
        Male       0.97      0.98      0.97       504

    accuracy                           0.97      1001
   macro avg       0.97      0.97      0.97      1001
weighted avg       0.97      0.97      0.97      1001


Confusion Matrix:
[[480  17]
 [ 12 492]]


In [10]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights
from responsibleai.feature_metadata import FeatureMetadata
categorical_features = []

In [11]:
feature_metadata = FeatureMetadata(categorical_features=categorical_features, dropped_features=[])
rai_insights = RAIInsights(model, train_data, test_data, target_feature, 'classification',
                               feature_metadata=feature_metadata)

In [12]:
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the label that they should have, and a list of 
                # strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')

In [ ]:
rai_insights.compute()

Causal Effects
Current Status: Generating Causal Effects.
Current Status: Finished generating causal effects.
Time taken: 0.0 min 1.5400000009435644e-05 sec
Counterfactual
Current Status: Generating 10 counterfactuals for 1001 samples


  6%|████▍                                                                           | 56/1001 [00:04<01:12, 12.95it/s]

In [ ]:
ResponsibleAIDashboard(rai_insights)